In [1]:
import pickle
import numpy as np
import pandas as pd
import re

seed = 2023

In [2]:
import os
os.chdir("../../")

In [3]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    histories_baseline_hyperparameter_search
)

/storage/ttseriotou/nlpsig-networks-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Rumours

In [4]:
%run notebooks/Rumours/load_sbert-embeddings.py

In [5]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


In [6]:
sbert_embeddings.shape

torch.Size([5568, 384])

# Baseline: FFN using signatures

In [7]:
dim_reduce_methods = ["umap"]
num_epochs = 100
batch=64
log_sig = True
log_signature_dimensions_and_sig_depths = [ (6,4)]#[(28, 2), (10, 3), (6, 4)]
hidden_dim_sizes = [[32,32]]#[[32,32], [64,64], [128,128], [256,256]]
dropout_rates = [0.2]#[0.5, 0.2, 0.1]
learning_rates = [1e-3]#[1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
patience=4
validation_metric = "f1"

In [8]:
ffn_logsignature_umap, best_ffn_logsignature_umap, _, __ = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings.numpy(),
    y_data=y_data,
    output_dim=output_dim,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=True,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    batch_size=batch,
    log_signature=log_sig,
    dim_reduce_methods=dim_reduce_methods,
    dimension_and_sig_depths=log_signature_dimensions_and_sig_depths, 
    path_indices=None,
    data_split_seed=123,
    split_ids= None, #torch.tensor(df_rumours['timeline_id'].astype(int)),
    split_indices = (df_rumours[df_rumours['set']=='train'].index, df_rumours[df_rumours['set']=='dev'].index, df_rumours[df_rumours['set']=='test'].index),
    k_fold=False,
    patience=patience,
    validation_metric=validation_metric,
    results_output=None,
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
Computing the path signature of the history for each item in the dataframe

























































































































100%|██████████| 5568/5568 [00:12<00:00, 456.12it/s]









100%|██████████| 1/1 [00:50<00:00, 50.72s/it]


100%|██████████| 1/1 [00:50<00:00, 50.72s/it]

100%|██████████| 1/1 [02:34<00:00, 154.74s/it]


[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' and feature...
[INFO] Adding 'time_diff' and feature...
[INFO] Adding 'timeline_index' feature...
Computing the path signature of the history for each item in the dataframe


100%|██████████| 5568/5568 [00:12<00:00, 459.00it/s]


In [9]:
ffn_logsignature_umap

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,...,gamma,k_fold,n_splits,batch_size,model_id,input_dim,dimension,sig_depth,method,log_signature
0,focal,0.532888,0.490978,"[0.6370370370370371, 0.3449197860962567]",0.245461,0.672598,0.553783,"[0.784037558685446, 0.3235294117647059]","(32, 32)",0.2,...,2,False,None,64,0.0,790,6,4,umap,True
0,focal,0.547188,0.511915,"[0.6431254695717504, 0.3807040417209908]",0.244329,0.626335,0.453581,"[0.7608200455580866, 0.14634146341463417]","(32, 32)",0.2,...,2,False,None,64,0.0,790,6,4,umap,True
0,focal,0.548141,0.520234,"[0.6359447004608294, 0.40452261306532666]",0.245648,0.701068,0.618058,"[0.7961165048543688, 0.44000000000000006]","(32, 32)",0.2,...,2,False,None,64,0.0,790,6,4,umap,True
0,focal,0.593899,0.565315,"[0.6767830045523521, 0.45384615384615384]",0.236428,0.661922,0.511304,"[0.782608695652174, 0.24]","(32, 32)",0.2,...,2,False,None,64,0.0,790,6,4,umap,True
0,focal,0.573880,0.555013,"[0.6466403162055335, 0.46338535414165666]",0.219582,0.679715,0.567781,"[0.7877358490566038, 0.3478260869565218]","(32, 32)",0.2,...,2,False,None,64,0.0,790,6,4,umap,True


In [12]:
best_ffn_logsignature_umap["valid_f1"].mean()

0.5409015615942542

In [10]:
best_ffn_logsignature_umap["f1"].mean()

0.5286908476697888

In [13]:
np.stack(best_ffn_logsignature_umap["f1_scores"]).mean(axis=0)

array([0.64790611, 0.40947559])